# Combination of all important functions
This is a summary of the important functions for constructing a network and solving it for the currents.   
First off there is the Construct function which takes the paths to the input data and builds a corresponding Networkx-Graph from it.  
Then there ist the Solve class which takes a network as input and can then construct the matrices and vectors needed to solve for the currents.   
In the end there is a snippet attached which can be used to orient the edges/current-directions. It takes a incidence-like matrix as input and changes the signs according to the potential landscape.

*Stand 21.07.23*

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy import sparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def Construct(Coo, Typ, Ener, J=0.003, T=300, Lam=0.2):
    #Read coordinates from file
    coords = list(map(tuple, np.genfromtxt(Coo, delimiter=" ")))
    #Build Graph
    F=nx.Graph()
    F.add_nodes_from(coords)
    #Read Attributes
    types = np.genfromtxt(Typ, delimiter=" ")
    energies = np.genfromtxt(Ener, delimiter=" ")
    #Set Attributes
    i=0
    for u in F.nodes():
        F.nodes[u]["pos"]=u
        F.nodes[u]["type"]=types[i]
        F.nodes[u]["potential"]=energies[i]
        i+=1  
    #Construct Edges
    F.add_edges_from(nx.geometric_edges(F, 1))
    
    ## Inizializing Marcus Rates ##
   
    for (u, v) in F.edges(): #die beiden Endpunkte u und v der Kante 
        #from settings file
        #J = attempt frequency (largest possible rate) =>  "maximum ti"
        #Lam =  Reorganization Energy = lambda = materials => l
        kbT=0.0000861801*T #eV
        H_Bar = 6.58264*(10**(-16)) # Planck in eV
        Qe = 1.602176634*(10**(-19)) # Electron charge
        
        deltaE=F.nodes[u]["potential"] - F.nodes[v]["potential"] ## v is start and u is end
        rate=2 * np.pi / H_Bar * np.abs(J)**2 * np.sqrt(1/(4 * Lam * kbT)) * np.exp(-((Lam + deltaE)**2)/(4* Lam * kbT))
        resistance =  deltaE / (Qe * rate )
        
        if deltaE <0: ## flow from high potential to low
            F.edges[u,v]['weight'] = resistance
        else: ## reorient edge
            # Could not just delete edge and add oposite as (Non-Di-)Graphs sort Edges by the sequence of nodes
            # Therefore I just add a negative sign to the resistor and thereby flipping the current direction
            F.edges[u,v]['weight'] = (-1) * resistance
    
    return F


#Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")

In [3]:
class Solve:
    def __init__(self, X):
        self.G=X
        self.N=self.G.order()
        self.nR=self.G.number_of_edges()
        
    def conductances(self):
        #Extract the values of the resistors from the graph and build a nR x nR matrix
        mat=sparse.spdiags(1/np.asarray(list((nx.get_edge_attributes(self.G, "weight").values()))), 0, self.nR, self.nR)
        return sparse.csc_matrix(mat)
    
    def incidence(self):
        #Builds the incidence matrix from the graph
        mat= np.transpose(nx.incidence_matrix(self.G, oriented=1)) #Beachte Transpose damit die Dimensionen der networkx funktion zum paper passen
        
        
        ## Siehe Codeschnipsel im ersten Beispiel falls die Orientierung ungünstig ist ##
        
        
        return mat
    
    def voltages(self):
        #Get the potential values from the nodes and build a vector
        vec=np.array(list(nx.get_node_attributes(self.G, "potential").values()))
        return vec

    
    def currents(self):
        #Combines the other functions to get the currents trough the resistors
        return - (self.conductances() @ self.incidence()) @ self.voltages()

    
#Solve(Gitter).currents()   

In [4]:
def Orientation(G):
    #Code Snippet that could be used for orienting the edges
    Mat=G.incidence()
    V=G.voltages()
    for x in range(np.shape(Mat)[0]):
        Y=[]
        for y in range(np.shape(Mat)[1]):
            if Mat[x,y] !=0:
                Y.append(y)
        Mat[x,Y[0]]=-np.sign(V[Y[0]]-V[Y[1]])
        Mat[x,Y[1]]=-Mat[x,Y[0]]
    return Mat

In [5]:
Gitter=Construct("coord_0.dat", "mol_types_0.dat", "site_energies_0.dat")
Setup=Solve(Gitter)
Setup.currents()

array([ 3.31321967e-08,  7.85273230e-08, -2.74166925e-11,  1.79547102e-08,
       -6.55817774e-10, -3.48554871e-09,  4.33005471e-08,  8.94586518e-08,
        3.15188284e-08, -6.47881356e-12,  8.74635482e-08,  4.93540764e-08,
       -4.04581937e-10,  2.95131612e-08,  3.36610146e-08, -5.46306294e-09,
       -3.51094122e-10,  9.43859418e-08, -1.87061156e-09,  4.11326481e-08,
       -5.77160145e-10,  9.38330100e-08, -5.22412401e-09, -4.15812718e-10,
        6.49794052e-08, -1.13245237e-11,  8.26656757e-08, -6.94857766e-09,
        1.40431708e-08,  9.21460226e-08, -3.73662175e-10,  5.10729462e-08,
       -3.61014936e-10,  4.40337328e-08, -1.26338555e-08, -1.14247265e-08,
       -6.68603138e-11,  9.55697657e-08, -6.69219463e-10,  3.44809054e-08,
       -1.29419211e-08, -6.95488187e-10,  7.38639340e-08, -6.47800978e-11,
       -4.26433491e-09,  5.89683906e-08,  1.97980807e-08])